In [1]:
from delphin.interfaces import ace
from delphin.mrs.simpledmrs import dump, dumps, serialize
from delphin.mrs import convert
from delphin.mrs.dmrx import loads
from delphin.extra.latex import dmrs_tikz_dependency
from collections import defaultdict

def parse_sentence2(sentence, grammar = '../ace-0.9.26/erg-1214-x86-64-0.9.26.dat'):
    p = ace.parse(grammar, sentence)
    top_result = p['RESULTS'][0]['MRS']
    dmrx = convert(p['RESULTS'][0]['MRS'], 'simplemrs', 'dmrx')
    dmrxx = loads(dmrx, single=True)
    return dmrxx.to_dict()



def make_links(links):
    d = defaultdict(dict)
    for link in links:
        d[link['from']][link['rargname']] = link['to']
    return d

def get_node(node, nodes):
    for n in nodes:
        if n['nodeid'] == node:
            return n
    else:
        return None
    
def full_parse(sentence):
    p = parse_sentence2(sentence)
    links = make_links(p['links'])
    nodes = p['nodes']
    return links, nodes

In [2]:
l, n = full_parse('don\'t put red blocks on blue blocks')

In [3]:
n

[{'lnk': {'from': 0, 'to': 5}, 'nodeid': 10000, 'predicate': 'pronoun_q'},
 {'lnk': {'from': 0, 'to': 5},
  'nodeid': 10001,
  'predicate': 'pron',
  'sortinfo': {'cvarsort': 'x', 'pers': '2', 'pt': 'zero'}},
 {'lnk': {'from': 0, 'to': 5},
  'nodeid': 10002,
  'predicate': 'neg',
  'sortinfo': {'cvarsort': 'i'}},
 {'lnk': {'from': 6, 'to': 9},
  'nodeid': 10003,
  'predicate': '_put_v_1',
  'sortinfo': {'cvarsort': 'e',
   'mood': 'indicative',
   'perf': '-',
   'prog': '-',
   'sf': 'comm',
   'tense': 'pres'}},
 {'lnk': {'from': 10, 'to': 20}, 'nodeid': 10004, 'predicate': 'udef_q'},
 {'lnk': {'from': 10, 'to': 13},
  'nodeid': 10005,
  'predicate': '_red_a_1',
  'sortinfo': {'cvarsort': 'e',
   'mood': 'indicative',
   'perf': '-',
   'prog': '-',
   'sf': 'prop',
   'tense': 'untensed'}},
 {'lnk': {'from': 14, 'to': 20},
  'nodeid': 10006,
  'predicate': '_block_n_of',
  'sortinfo': {'cvarsort': 'x', 'ind': '+', 'num': 'pl', 'pers': '3'}},
 {'lnk': {'from': 21, 'to': 23},
  'nodei

In [4]:
get_node(10003, n)

{'lnk': {'from': 6, 'to': 9},
 'nodeid': 10003,
 'predicate': '_put_v_1',
 'sortinfo': {'cvarsort': 'e',
  'mood': 'indicative',
  'perf': '-',
  'prog': '-',
  'sf': 'comm',
  'tense': 'pres'}}

In [5]:
def get_pos(string):
    parts = string.strip('_').split('_')
    try:
        return parts[1]
    except IndexError:
        return ''

def find_verb(links, nodes):
    root = links[0][None]
    curr_link = root
    while get_pos(get_node(curr_link, nodes)['predicate']) != 'v':
        curr_link = links[curr_link]['ARG1']
    return get_node(curr_link, nodes)
    
def find_rel(links, nodes):
    verb = find_verb(links, nodes)
    nodeid = verb['nodeid']
    rel = links[nodeid]['ARG3']
    return get_node(rel, nodes)

def find_open_class(links, nodes, arg='ARG1'):
    rel = find_rel(links, nodes)
    obj = []
    root = links[rel['nodeid']][arg]
    obj.append(get_node(root, nodes))
    for key,value in links.items():
        try:
            link = value['ARG1']
        except KeyError:
            continue
        if root == link:
            node = get_node(key, nodes)
            if get_pos(node['predicate']) != 'p':
                obj.append(get_node(key, nodes))
            
    return obj

In [6]:
find_open_class(l, n)

[{'lnk': {'from': 14, 'to': 20},
  'nodeid': 10006,
  'predicate': '_block_n_of',
  'sortinfo': {'cvarsort': 'x', 'ind': '+', 'num': 'pl', 'pers': '3'}},
 {'lnk': {'from': 10, 'to': 13},
  'nodeid': 10005,
  'predicate': '_red_a_1',
  'sortinfo': {'cvarsort': 'e',
   'mood': 'indicative',
   'perf': '-',
   'prog': '-',
   'sf': 'prop',
   'tense': 'untensed'}}]

In [7]:
find_open_class(l, n, arg='ARG2')

[{'lnk': {'from': 29, 'to': 35},
  'nodeid': 10010,
  'predicate': '_block_n_of',
  'sortinfo': {'cvarsort': 'x', 'ind': '+', 'num': 'pl', 'pers': '3'}},
 {'lnk': {'from': 24, 'to': 28},
  'nodeid': 10009,
  'predicate': '_blue_a_1',
  'sortinfo': {'cvarsort': 'e',
   'mood': 'indicative',
   'perf': '-',
   'prog': '-',
   'sf': 'prop',
   'tense': 'untensed'}}]

In [8]:
def get_tripple(links, nodes):
    rel = find_rel(links, nodes)
    obj1 = find_open_class(links, nodes, arg='ARG1')
    obj2 = find_open_class(links, nodes, arg='ARG2')
    return rel, obj1, obj2

In [9]:
def sent_to_tripple(sentence):
    links, nodes = full_parse(sentence)
    return get_tripple(links, nodes)

In [10]:
sent_to_tripple('put red blocks on blue blocks')

({'lnk': {'from': 15, 'to': 17},
  'nodeid': 10006,
  'predicate': '_on_p',
  'sortinfo': {'cvarsort': 'e',
   'mood': 'indicative',
   'perf': '-',
   'prog': '-',
   'sf': 'prop',
   'tense': 'untensed'}},
 [{'lnk': {'from': 8, 'to': 14},
   'nodeid': 10005,
   'predicate': '_block_n_of',
   'sortinfo': {'cvarsort': 'x', 'ind': '+', 'num': 'pl', 'pers': '3'}},
  {'lnk': {'from': 4, 'to': 7},
   'nodeid': 10004,
   'predicate': '_red_a_1',
   'sortinfo': {'cvarsort': 'e',
    'mood': 'indicative',
    'perf': '-',
    'prog': '-',
    'sf': 'prop',
    'tense': 'untensed'}}],
 [{'lnk': {'from': 23, 'to': 29},
   'nodeid': 10009,
   'predicate': '_block_n_of',
   'sortinfo': {'cvarsort': 'x', 'ind': '+', 'num': 'pl', 'pers': '3'}},
  {'lnk': {'from': 18, 'to': 22},
   'nodeid': 10008,
   'predicate': '_blue_a_1',
   'sortinfo': {'cvarsort': 'e',
    'mood': 'indicative',
    'perf': '-',
    'prog': '-',
    'sf': 'prop',
    'tense': 'untensed'}}])

In [11]:
sent_to_tripple('put red blocks on blue blocks')

({'lnk': {'from': 15, 'to': 17},
  'nodeid': 10006,
  'predicate': '_on_p',
  'sortinfo': {'cvarsort': 'e',
   'mood': 'indicative',
   'perf': '-',
   'prog': '-',
   'sf': 'prop',
   'tense': 'untensed'}},
 [{'lnk': {'from': 8, 'to': 14},
   'nodeid': 10005,
   'predicate': '_block_n_of',
   'sortinfo': {'cvarsort': 'x', 'ind': '+', 'num': 'pl', 'pers': '3'}},
  {'lnk': {'from': 4, 'to': 7},
   'nodeid': 10004,
   'predicate': '_red_a_1',
   'sortinfo': {'cvarsort': 'e',
    'mood': 'indicative',
    'perf': '-',
    'prog': '-',
    'sf': 'prop',
    'tense': 'untensed'}}],
 [{'lnk': {'from': 23, 'to': 29},
   'nodeid': 10009,
   'predicate': '_block_n_of',
   'sortinfo': {'cvarsort': 'x', 'ind': '+', 'num': 'pl', 'pers': '3'}},
  {'lnk': {'from': 18, 'to': 22},
   'nodeid': 10008,
   'predicate': '_blue_a_1',
   'sortinfo': {'cvarsort': 'e',
    'mood': 'indicative',
    'perf': '-',
    'prog': '-',
    'sf': 'prop',
    'tense': 'untensed'}}])

In [12]:
full_parse('have red blocks on blue blocks')

(defaultdict(dict,
             {0: {None: 10002},
              10000: {'RSTR': 10001},
              10002: {'ARG1': 10001, 'ARG2': 10005},
              10003: {'RSTR': 10005},
              10004: {'ARG1': 10005},
              10006: {'ARG1': 10002, 'ARG2': 10009},
              10007: {'RSTR': 10009},
              10008: {'ARG1': 10009}}),
 [{'lnk': {'from': 0, 'to': 30}, 'nodeid': 10000, 'predicate': 'pronoun_q'},
  {'lnk': {'from': 0, 'to': 30},
   'nodeid': 10001,
   'predicate': 'pron',
   'sortinfo': {'cvarsort': 'x', 'pers': '2', 'pt': 'zero'}},
  {'lnk': {'from': 0, 'to': 4},
   'nodeid': 10002,
   'predicate': '_have_v_1',
   'sortinfo': {'cvarsort': 'e',
    'mood': 'indicative',
    'perf': '-',
    'prog': '-',
    'sf': 'comm',
    'tense': 'pres'}},
  {'lnk': {'from': 5, 'to': 15}, 'nodeid': 10003, 'predicate': 'udef_q'},
  {'lnk': {'from': 5, 'to': 8},
   'nodeid': 10004,
   'predicate': '_red_a_1',
   'sortinfo': {'cvarsort': 'e',
    'mood': 'indicative',
    'pe

In [13]:
full_parse('make sure to put red blocks on blue blocks')

KeyboardInterrupt: 

In [14]:
full_parse('no, now you can\'t put that in the tower because red blocks go on blue blocks')

KeyboardInterrupt: 

In [15]:
parse_sentence2('no, now you can\'t put than in the tower because red blocks go on blue blocks')

IndexError: list index out of range

In [16]:
sent_to_tripple('red blocks go on blue blocks')

KeyError: 'ARG3'